## Creating a mesh

We can also use the Splatter wrapper class to take an existing nerfstudio model and create a mesh!
1. **mesh:** creates a mesh via TSDF fusion

2. **query_mesh:** uses the trained model to query the mesh and returns a similarity map

3. **plot_mesh:** enables plotting of mesh features



In [1]:
import os, sys
from pathlib import Path
from collab_splats.wrapper import Splatter, SplatterConfig
import pyvista as pv

pv.start_xvfb()


/opt/conda/envs/nerfstudio/lib/python3.10/site-packages/pyvista/plotting/utilities/xvfb.py:48: PyVistaDeprecationWarning: This function is deprecated and will be removed in future version of PyVista. Use vtk-osmesa instead.
  warnings.warn(


Set paths to the file for running splats

In [2]:
base_dir = Path('/workspace/fieldwork-data/')
session_dir = base_dir / "rats/2024-07-11/SplatsSD"

# Make the configuration 
splatter_config = SplatterConfig(
    file_path=session_dir / "C0119.MP4",
    method='rade-features',
    frame_proportion=0.25, # Use 25% of the frames within the video (or default to minimum 300 frames)
)

# Initialize the Splatter class
splatter = Splatter(splatter_config)

# Call these to populate the splatter with paths (probably a better way to do this --> maybe save out config)
splatter.preprocess()
splatter.extract_features()

transforms.json already exists at /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/preproc/transforms.json
To rerun preprocessing, set overwrite=True
Output already exists for rade-features
To rerun feature extraction, set overwrite=True


### Create a mesh

We can create a mesh by calling the ```mesh()``` method. Under the hood, this runs TSDF fusion creating an integrated volume. 

In [3]:
mesher_kwargs = {
    'depth_name': "median_depth",
    'depth_trunc': 1.0,
    'normals_name': "normals",
    'features_name': "distill_features", 
    'sdf_trunc': 0.03,
    'clean_repair': True,
}

splatter.mesh(
    mesher_type="Open3DTSDFFusion",
    mesher_kwargs=mesher_kwargs,
    # overwrite=True
)


Available runs:
[0] 2025-07-11_171420


### Plot the mesh!

We can use the splatter function ```plot_mesh``` to visualize given attributes of the mesh. The inherent attributes are RGB and Normals

In [9]:
splatter.plot_mesh(attribute="RGB")

Number of points: 102294
Number of cells: 200193
Bounds: BoundsTuple(x_min=-0.5049999952316284, x_max=0.7450000047683716, y_min=-0.125, y_max=1.4850000143051147, z_min=-1.0152238607406616, z_max=0.9950358867645264)


Widget(value='<iframe src="http://localhost:34321/index.html?ui=P_0x76db4943f460_5&reconnect=auto" class="pyvi…

### Using semantic queries 

The mesh contains semantic features which we can query via positive and negative prompts. The goal of this is to find points that are more similar to the positive prompts compared to the negative prompts

In [4]:
similarity = splatter.query_mesh(
    positive_queries=["tree"],
    negative_queries=["ground", "leaves"],
)

Loading model from /workspace/fieldwork-data/rats/2024-07-11/environment/C0119/rade-features/2025-07-11_171420/config.yml
[Taichi] version 1.7.3, llvm 15.0.4, commit 5ec301be, linux, python 3.10.18


[I 07/23/25 20:43:35.153 203920] [shell.py:_shell_pop_print@23] Graphical python shell detected, using wrapped sys.stdout


ConstructorError: while constructing a Python object
cannot find module 'ns_extension.utils.trainer_config' (No module named 'ns_extension')
  in "<unicode string>", line 1, column 1:
    !!python/object:ns_extension.uti ... 
    ^

Plot similarity maps

In [54]:
splatter.plot_mesh(attribute=similarity, rgb=False)

Number of points: 116178
Number of cells: 215841
Bounds: BoundsTuple(x_min=-0.5249999761581421, x_max=0.7450000047683716, y_min=-0.125, y_max=1.4850000143051147, z_min=-1.0149999856948853, z_max=1.0214753150939941)


Widget(value='<iframe src="http://localhost:34453/index.html?ui=P_0x76d499a8f5e0_17&reconnect=auto" class="pyv…

### Lets clean up the mesh

### Fix mesh multistep

#### Testing meshlib

In [10]:
cleaned_mesh = pv.read("test.ply")
original_mesh = pv.read(splatter.config['mesh_info']['mesh'])

In [ ]:
from collab_splats.utils.mesh import normals2vertex, features2vertex

normals = normals2vertex(cleaned_mesh.points, original_mesh.points, original_mesh.point_data['Normals'])
rgb = features2vertex(cleaned_mesh.points, original_mesh.points, original_mesh.point_data['RGB'])
tree_features = features2vertex(cleaned_mesh.points, original_mesh.points, similarity)

cleaned_mesh.point_data['Normals'] = normals
cleaned_mesh.point_data['RGB'] = np.asarray(rgb).astype(np.uint8)
cleaned_mesh.point_data['tree'] = tree_features

In [17]:
save_pyvista_to_open3d_ply(cleaned_mesh, "example_mesh")

Adding normals
[Open3D WARNING] Write geometry::TriangleMesh failed: unknown file extension.
Saved: example_mesh
Saved: example_mesh_tree.ply


In [12]:
import pyvista as pv
import open3d as o3d
import numpy as np
import os

def save_pyvista_to_open3d_ply(pv_mesh, filename_base):
    """
    Save a PyVista mesh to Open3D-compatible PLY format.
    
    Parameters:
        pv_mesh (pyvista.PolyData): Input mesh with point data.
        filename_base (str): Base filename without extension.
    """
    # Extract basic geometry
    points = np.asarray(pv_mesh.points)
    faces = pv_mesh.faces.reshape((-1, 4))[:, 1:]  # triangle faces assumed

    # Prepare base Open3D mesh
    base_mesh = o3d.geometry.TriangleMesh()
    base_mesh.vertices = o3d.utility.Vector3dVector(points)
    base_mesh.triangles = o3d.utility.Vector3iVector(faces)

    # Handle RGB
    if "RGB" in pv_mesh.point_data:
        rgb = np.asarray(pv_mesh.point_data["RGB"])
        if rgb.max() > 1.0:
            rgb = rgb / 255.0
        base_mesh.vertex_colors = o3d.utility.Vector3dVector(rgb)
    
    # Handle normals
    if "Normals" in pv_mesh.point_data:
        print (f"Adding normals")
        normals = np.asarray(pv_mesh.point_data["Normals"])
        base_mesh.vertex_normals = o3d.utility.Vector3dVector(normals)

    # Save main mesh
    main_path = f"{filename_base}"
    o3d.io.write_triangle_mesh(main_path, base_mesh, write_ascii=True)
    print(f"Saved: {main_path}")

    # Save other point data fields in separate files
    for key in pv_mesh.point_data.keys():
        if key in ["RGB", "Normals"]:
            continue
        
        attr = np.asarray(pv_mesh.point_data[key])
        if attr.ndim == 1:
            attr = attr[:, np.newaxis]
        if attr.shape[1] > 3:
            print(f"Skipping {key}: more than 3 channels (shape={attr.shape})")
            continue

        # Normalize and pad to RGB
        color_data = np.zeros((len(attr), 3))
        norm_attr = attr.astype(np.float64)
        if np.max(norm_attr) > 0:
            norm_attr /= np.max(norm_attr)
        color_data[:, :attr.shape[1]] = norm_attr

        # Build new mesh
        custom_mesh = o3d.geometry.TriangleMesh()
        custom_mesh.vertices = o3d.utility.Vector3dVector(points)
        custom_mesh.triangles = o3d.utility.Vector3iVector(faces)
        custom_mesh.vertex_colors = o3d.utility.Vector3dVector(color_data)

        # Save it
        out_path = f"{filename_base}_{key}.ply"
        o3d.io.write_triangle_mesh(out_path, custom_mesh, write_ascii=True)
        print(f"Saved: {out_path}")


In [18]:
import open3d as o3d

mesh = o3d.io.read_triangle_mesh("example_mesh.ply")

# o3d.visualization.draw_geometries([mesh])

In [12]:
import pyvista as pv

vis_mesh = pv.read(splatter.config['mesh_info']['mesh'])

vis_mesh.plot(scalars='RGB', rgb=True)


Widget(value='<iframe src="http://localhost:34321/index.html?ui=P_0x76db4943f310_8&reconnect=auto" class="pyvi…